In [1]:
import random
import multiprocessing
import cProfile


In [2]:
def pi_serial(samples):
    hits = 0
    for i in range(samples):
        x = random.uniform(-1.0, 1.0)
        y = random.uniform(-1.0, 1.0)
        if x**2 + y**2 <= 1:
            hits += 1
    return 4.0 * hits / samples

In [3]:
def sample():
    x = random.uniform(-1.0, 1.0)
    y = random.uniform(-1.0, 1.0)
    if x**2 + y**2 <= 1:
        return 1
    else:
        return 0

def pi_apply_async(samples):
    pool = multiprocessing.Pool()
    results_async = [pool.apply_async(sample) for i in range(samples)]
    hits = sum(r.get() for r in results_async)
    return 4.0 * hits / samples

In [4]:
def sample_multiple(samples_partial):
    return sum(sample() for i in range(samples_partial))

def pi_apply_async_chunked(samples, n_tasks):
    chunk_size = samples // n_tasks
    pool = multiprocessing.Pool()
    results_async = [pool.apply_async(sample_multiple, (chunk_size,))
                     for i in range(n_tasks)]
    hits = sum(r.get() for r in results_async)
    return 4.0 * hits / samples


In [5]:
%%timeit
pi_serial(100000)

97.4 ms ± 21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
pi_apply_async(100000)

In [ ]:
%timeit
pi_apply_async_chunked(100000, 10)

In [ ]:
# Profile pi_serial()
print("Serial profiling")
cProfile.run('pi_serial(100000)')

print("Apply async profiling")
# Profile pi_apply_async()
cProfile.run('pi_apply_async(1000000)')

print("Apply async chuncked profiling")
# Profile pi_apply_async_chunked()
cProfile.run('pi_apply_async_chunked(100000, 10)')